# Criando Catalog
- O "Catalog" é o nível mais alto de organização no Unity Catalog.
- Ele representa uma coleção de schemas (databases) e tabelas dentro de um mesmo domínio lógico.
- É como uma "instância" de dados dentro do Databricks, usada para separar ambientes (ex: DEV, QAS, PRD)
- ou domínios (ex: comercial, financeiro, industrial).
- O comando abaixo cria o catálogo "demo_catalog" se ele ainda não existir.

In [0]:
%sql
create catalog if not exists demo_catalog
comment 'Catalog de estudo'

# Criando Schema (database) no Catalog
- Dentro de cada Catalog, criamos Schemas (ou Databases) para organizar as tabelas.
- O Schema "bronze" segue a arquitetura medallion (Bronze, Silver, Gold).
- Aqui armazenaremos os dados brutos, exatamente como foram ingeridos das fontes originais.
- O comando cria o schema "demo_catalog.bronze" caso ele não exista.

In [0]:
%sql
create schema if not exists demo_catalog.bronze
comment 'Schema Bronze para os dados brutos'

In [0]:
%sql
create schema if not exists demo_catalog.silver
comment 'Schema Silver para dados tratados'

In [0]:
%sql
create schema if not exists demo_catalog.gold
comment 'Schema Gold para dados agregados que serão utilizados pelas áreas de negócio'

# Criando Tabelas, Views, Volumes, Functions e Models

Dentro de cada **Schema** do Unity Catalog, podemos criar diferentes tipos de **objetos de dados e processamento**.  
Esses objetos formam a base do Lakehouse e representam o ciclo completo da ingestão ao consumo analítico.

---


Essa estrutura garante:

- **Isolamento lógico** entre ambientes (DEV, QAS, PRD) ou domínios (financeiro, logística, comercial);
- **Governança centralizada** de acessos e permissões (ACLs);
- **Auditoria e lineage automáticos**, registrando quem criou, modificou e consultou os objetos;
- **Consistência e versionamento** de dados.

---

## Tabelas (Tables)

As **tabelas** são os principais objetos de armazenamento de dados.  
Elas podem conter dados **estruturados** (como colunas e linhas) e são integradas ao Delta Lake para garantir confiabilidade e performance.

### Tipos de tabelas:
- **Managed Tables**: o Databricks gerencia tanto os metadados quanto a localização física dos arquivos.  
- **External Tables**: a tabela referencia arquivos já existentes em um local externo (por exemplo, um caminho no Data Lake).

Essas tabelas herdam automaticamente as políticas de segurança e controle de acesso definidas no Unity Catalog, além de participar do lineage e da auditoria.

---

## Views

As **Views** são representações lógicas dos dados, baseadas em consultas SQL sobre tabelas existentes.  
Elas não armazenam fisicamente os dados, mas permitem criar **camadas virtuais** com regras de negócio e filtros aplicados.

### Tipos de views:
- **Views comuns**: dependem do contexto de execução, sendo recalculadas a cada consulta.  
- **Materialized Views**: armazenam os resultados para acelerar consultas frequentes, ideais para grandes volumes de dados.

As Views são amplamente usadas na camada **Silver** e **Gold** do modelo Medallion para garantir padronização e reuso.

---

## Volumes

Os **Volumes** são áreas de armazenamento gerenciadas pelo Unity Catalog dentro do Data Lake.  
Servem para guardar **arquivos brutos ou semiestruturados**, como CSV, JSON, Parquet, imagens ou logs.

Eles oferecem:
- Controle de acesso via ACLs (Access Control Lists);
- Governança unificada sobre arquivos não estruturados;
- Integração com workflows de ingestão.

Os Volumes são ideais para a **camada Bronze**, onde os dados ainda estão em formato original.

---

## Functions

As **Functions** (ou UDFs – User Defined Functions) permitem encapsular e reutilizar lógicas de transformação.  
Podem ser criadas diretamente dentro de um Schema e chamadas por múltiplos notebooks, jobs ou dashboards.

Elas são úteis para:
- Padronizar cálculos complexos;
- Reduzir repetição de lógica SQL;
- Facilitar governança e versionamento de transformações.

O Unity Catalog garante que as funções tenham **controle de versão**, **segurança** e **auditoria** integrados.

---

## Models

Os **Models** representam modelos de Machine Learning registrados e versionados dentro do Unity Catalog.  
Eles são integrados ao **MLflow**, permitindo rastrear:
- Versões de modelos;
- Autores e datasets usados no treinamento;
- Metadados e métricas de performance.

Essa integração traz governança também para o ciclo de vida de **Machine Learning (ML Governance)**, garantindo que todo o processo seja rastreável e auditável.

---

## Resumo Geral

| Objeto | Finalidade Principal | Tipo de Dado | Governança | Camada Indicada |
|---------|----------------------|---------------|-------------|------------------|
| **Table** | Armazenar dados estruturados | Estruturado (Delta) | Total | Bronze, Silver, Gold |
| **View** | Criar camada lógica de consulta | Estruturado | Parcial | Silver, Gold |
| **Volume** | Armazenar arquivos brutos | Semi ou não estruturado | Total | Bronze |
| **Function** | Reutilizar lógica de cálculo | N/A | Total | Todas |
| **Model** | Registrar e versionar modelos de ML | Modelo treinado | Total | ML |

---

## Boas Práticas

- Utilize sempre o padrão `<catalog>.<schema>.<objeto>` para nomear.  
- Documente os objetos com `COMMENT` para manter a governança viva.  
- Prefira **Delta Tables** para tabelas gerenciadas.  
- Mantenha **camadas bem definidas**: Bronze (bruto), Silver (tratado), Gold (agregado).  
- Centralize a segurança e o acesso no Unity Catalog.  
- Crie funções e views para abstrair a lógica de negócio e manter o modelo consistente.



In [0]:
%sql
create table demo_catalog.bronze.sales (
  sale_id int,
  product string,
  quantity int,
  price double,
  sale_date date
)
using delta
comment 'Dados brutos de vendas'

In [0]:
%sql
insert into demo_catalog.bronze.sales (sale_id, product, quantity, price, sale_date)
values
-- Grupo 1: Notebooks
(1, 'Notebook Dell', 1, 5500.00, date'2025-10-01'),
(2, 'Notebook Dell', 2, 5200.00, date'2025-10-02'),
(3, 'Notebook Dell', 1, 5300.00, date'2025-10-03'),
(4, 'Notebook Apple', 3, 10000.00, date'2025-10-04'),
(5, 'Notebook Apple', 3, 10000.00, date'2025-10-05'),
-- Grupo 2: Monitor
(6, 'Monitor Samsung', 2, 1200.00, date'2025-10-06'),
(7, 'Monitor Samsung', 1, 1150.00, date'2025-10-07'),
(8, 'Monitor LG', 3, 1100.00, date'2025-10-08'),
(9, 'Monitor LG', 2, 1050.00, date'2025-10-09'),
(10, 'Monitor Dell', 1, 1300.00, date'2025-10-10'),
-- Grupo 3: Headset
(11, 'Headset Logitech', 2, 350.00, date'2025-10-11'),
(12, 'Headset Logitech', 1, 340.00, date'2025-10-12'),
(13, 'Headset HyperX', 3, 400.00, date'2025-10-13'),
(14, 'Headset HyperX', 2, 390.00, date'2025-10-14'),
(15, 'Headset Sony', 1, 420.00, date'2025-10-15')

In [0]:
%sql
select * from demo_catalog.bronze.sales

In [0]:
%sql
create or replace view demo_catalog.bronze.sales_summary as
select
  product,
  sum(quantity) as total_quantity,
  sum(quantity * price) as revenue
from
  demo_catalog.bronze.sales
group by product;

# Criando Volume(arquivos)

In [0]:
%sql
create volume demo_catalog.bronze.raw_files
comment 'Volume para armazenar os arquivos brutos ou de ingestão inicial'